In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
HEIGHT = 160
WIDTH = 90

In [2]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-
from __future__ import division, print_function, unicode_literals
import tensorflow as tf
from time import time
from keras.utils import np_utils#one hot
import os
from skimage import data
from skimage import transform
import skimage
import numpy as np

# Download the dataset


def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory) 
                  if os.path.isdir(os.path.join(data_directory,d))]
    labels=[]
    images=[]
    for d in directories:
        label_directory = os.path.join(data_directory,d)
        file_names = [os.path.join(label_directory,f)
                     for f in os.listdir(label_directory)
                     if f.endswith('.jpg')]
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(str(d))
    return images, labels

Using TensorFlow backend.


In [3]:
ROOT_PATH = '/home/tang/targa/exp3/'
train_data_directory = os.path.join(ROOT_PATH, "padding_160*90_train")
#validation_data_directory = os.path.join(ROOT_PATH, "16_8_1568_train") 16_8_4448_train  16_8_1248_train 
validation_data_directory = os.path.join(ROOT_PATH, "16_8_1248_train")
test_data_directory = os.path.join(ROOT_PATH, "padding_160*90_val")

In [4]:
train_images,train_labels = load_data(train_data_directory)
test_images,test_labels = load_data(test_data_directory)
validation_images,validation_labels = load_data(validation_data_directory)

In [5]:
#encoding trY teY
from numpy import array
#from numpy import argmax
#from keras.utils import to_categorical
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example

#train_labels = np_utils.to_categorical(train_labels,num_classes = None)
#test_labels = np_utils.to_categorical(test_labels,num_classes = None)
trL,teL,valL=array(train_labels),array(test_labels),array(validation_labels)
label_encoder = LabelEncoder()

In [6]:
integer_encoded_trL = label_encoder.fit_transform(trL)
integer_encoded_teL = label_encoder.fit_transform(teL)
integer_encoded_valL = label_encoder.fit_transform(valL)

In [7]:
trainLabels = np_utils.to_categorical(integer_encoded_trL,num_classes = 32)
testLabels = np_utils.to_categorical(integer_encoded_teL,num_classes = 32)
validationLabels = np_utils.to_categorical(integer_encoded_valL,num_classes = 32)

In [10]:
########convert rgb images into black and white
#convert images into size of vector
from skimage.color import rgb2gray

trainImages = rgb2gray(np.array(train_images))
testImages = rgb2gray(np.array(test_images))
valImages = rgb2gray(np.array(validation_images))
train_flat_images = np.reshape(trainImages,(len(trainImages),HEIGHT*WIDTH))
test_flat_images = np.reshape(testImages,(len(testImages),HEIGHT*WIDTH))
val_flat_images = np.reshape(valImages,(len(valImages),16*8))

In [11]:
train_flat_images[0].shape

(14400,)

In [12]:
print('train images shape:\t\t{}'.format(train_flat_images.shape))

print('test images shape:\t\t{}'.format(test_flat_images.shape))

print('val images shape:\t\t{}'.format(val_flat_images.shape))

train images shape:		(1246, 14400)
test images shape:		(411, 14400)
val images shape:		(1246, 128)


In [13]:
batch_size = 100
n_batch = len(trainLabels)//batch_size
lr = tf.Variable(0.001,dtype = tf.float32)

In [23]:
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean) # 平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histgram',var) # 直方图

In [24]:

#命名空间
with tf.name_scope('input'):
    #定义两个placeholder 
    #把批次的图片传进去，开始是100张一次的传送，直到最后
    x = tf.placeholder(tf.float32,[None,WIDTH*HEIGHT],name='x-input')
    y = tf.placeholder(tf.float32,[None,32],name='y-input')
    #定义一个dropout的值
    keep_prob = tf.placeholder(tf.float32,name = 'non-dropout-rate')
with tf.name_scope('layer'):
    with tf.name_scope('W1'):
        #第一层
        W1 = tf.Variable(tf.truncated_normal([WIDTH*HEIGHT,50],stddev=0.1),name='W1')#改成截断正太分布
    with tf.name_scope('b1'):
        
        b1 = tf.Variable(tf.zeros([50])+0.1,name = 'b1')
    with tf.name_scope('out1'):
        
        output1 = tf.matmul(x,W1) + b1
    with tf.name_scope('L1'):
        L1 = tf.nn.tanh(output1)
    with tf.name_scope('L1-drop'):
        #L1为第一层的输出，keep_prob为1的话就是不丢失，如果为0的话就是全丢，如果为0.5的话就是丢失50%的神经元
        L1_drop = tf.nn.dropout(L1,keep_prob)

    '''
    with tf.name_scope('W2'):
        #第二层
        W2 = tf.Variable(tf.truncated_normal([1000,2000],stddev=0.1),name='W2')
    with tf.name_scope('b2'):
        b2 = tf.Variable(tf.zeros([2000])+0.2,name='b2')
    with tf.name_scope('output2'):
        output2 = tf.matmul(L1_drop,W2)+b2
    with tf.name_scope('L2'):
        
        L2 = tf.nn.tanh(output2)
    with tf.name_scope('L2_drop'):
        #L1为第一层的输出，keep_prob为1的话就是不丢失，如果为0的话就是全丢，如果为0.5的话就是丢失50%的神经元
        L2_drop = tf.nn.dropout(L2,keep_prob)

    with tf.name_scope('W3'):
        #第三层
        W3 = tf.Variable(tf.truncated_normal([2000,1000],stddev=0.1),name='W3')
    with tf.name_scope('b3'):
        b3 = tf.Variable(tf.zeros([1000])+0.1,name='b3')
    with tf.name_scope('output3'):   
        output3 = tf.matmul(L2_drop,W3)+b3
    with tf.name_scope('L3'):
        L3 = tf.nn.tanh(output3)
    with tf.name_scope('L3-drop'):
        #L1为第一层的输出，keep_prob为1的话就是不丢失，如果为0的话就是全丢，如果为0.5的话就是丢失50%的神经元
        L3_drop = tf.nn.dropout(L3,keep_prob)
        '''
    with tf.name_scope('W4'):
    #第四层
        W4 = tf.Variable(tf.truncated_normal([50,32],stddev=0.1))
    with tf.name_scope('b4'):        
        b4 = tf.Variable(tf.zeros([32])+0.1)
    with tf.name_scope('output4'):
        output4 = tf.matmul(L1_drop,W4)+ b4
    with tf.name_scope('softmax'):
        #预测分类
        prediction = tf.nn.softmax(output4)
with tf.name_scope('loss'):
    #定义一个损失函数
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits = prediction))#联合交叉熵
    tf.summary.scalar('cross_entropy',loss)
    #loss = tf.reduce_mean(tf.square(y-prediction))
with tf.name_scope('train'):
    #定义一个方法来收敛神经网络，使用梯度下降法,学习率=0.1
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
    #train_step = tf.train.AdamOptimizer(0.1).minimize(loss)
#初始化
init = tf.global_variables_initializer()
with tf.name_scope('accuracy'):
    with tf.name_scope('correct-prediction'):
        #定义一个求准确率的方法
        #如果有两个相同的就会返回TRUE，不相同就返回false，然后就依次存入correct_prediction
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中的最大值
    with tf.name_scope('accuracy'):    
        #求准确率
        #首先把bool值转化成32位的浮点数，然后求平均值
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)

In [25]:
merged = tf.summary.merge_all()

In [26]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [29]:
with tf.Session() as sess:
    sess.run(init)
    #路径和文件名
    #write = tf.summary.FileWriter('logs/twoLayerSmallSize/4448',sess.graph)
    #train_writer = tf.summary.FileWriter('logs/paddingsmall16_8/train',sess.graph)
    #test_writer = tf.summary.FileWriter('logs/paddingsmall16_8/test',sess.graph)
    for epoch in range(10000000):#把所有的图片训练20次
        #一批一批的迭代图片，一次迭代100张，一共运行n_batch次
        for batch in range(n_batch):
            #每次传入一百张图片
            batch_xs,batch_ys = next_batch(batch_size,train_flat_images,trainLabels)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        #write.add_summary(summary,epoch)
        test_acc = sess.run(accuracy,feed_dict={x:test_flat_images,y:testLabels,keep_prob:1.0})
        #test_writer.add_summary(test_acc,epoch)
        train_acc = sess.run(accuracy,feed_dict={x:train_flat_images,y:trainLabels,keep_prob:1.0})
        #train_writer.add_summary(train_acc,epoch)
        if epoch%200 == 0:
            
            print('epoch=' + str(epoch) + ',train accuracy='+ str(train_acc)+',test accuracy=' + str(test_acc))

epoch=0,train accuracy=0.016853932,test accuracy=0.01459854
epoch=200,train accuracy=0.09470305,test accuracy=0.07785888
epoch=400,train accuracy=0.35313,test accuracy=0.2189781
epoch=600,train accuracy=0.5730337,test accuracy=0.32846716
epoch=800,train accuracy=0.6621188,test accuracy=0.3941606
epoch=1000,train accuracy=0.6813804,test accuracy=0.39659366
epoch=1200,train accuracy=0.7110754,test accuracy=0.42335767
epoch=1400,train accuracy=0.71910113,test accuracy=0.4379562
epoch=1600,train accuracy=0.7247191,test accuracy=0.44768855
epoch=1800,train accuracy=0.73113966,test accuracy=0.4452555
epoch=2000,train accuracy=0.73354733,test accuracy=0.45742092
epoch=2200,train accuracy=0.752809,test accuracy=0.46958637
epoch=2400,train accuracy=0.7600321,test accuracy=0.46958637
epoch=2600,train accuracy=0.7608347,test accuracy=0.47201946
epoch=2800,train accuracy=0.76324236,test accuracy=0.4647202
epoch=3000,train accuracy=0.7656501,test accuracy=0.4647202
epoch=3200,train accuracy=0.76725

epoch=26600,train accuracy=0.83707863,test accuracy=0.5060827
epoch=26800,train accuracy=0.83707863,test accuracy=0.5060827
epoch=27000,train accuracy=0.83707863,test accuracy=0.50121653
epoch=27200,train accuracy=0.83707863,test accuracy=0.50364965
epoch=27400,train accuracy=0.83707863,test accuracy=0.5060827
epoch=27600,train accuracy=0.83707863,test accuracy=0.5060827
epoch=27800,train accuracy=0.83707863,test accuracy=0.5060827
epoch=28000,train accuracy=0.8378812,test accuracy=0.50851583
epoch=28200,train accuracy=0.8378812,test accuracy=0.50851583
epoch=28400,train accuracy=0.8378812,test accuracy=0.50364965
epoch=28600,train accuracy=0.8378812,test accuracy=0.50851583
epoch=28800,train accuracy=0.8378812,test accuracy=0.50851583
epoch=29000,train accuracy=0.8386838,test accuracy=0.5109489
epoch=29200,train accuracy=0.8386838,test accuracy=0.5060827
epoch=29400,train accuracy=0.8386838,test accuracy=0.50851583
epoch=29600,train accuracy=0.8386838,test accuracy=0.5060827
epoch=298

epoch=53400,train accuracy=0.8507223,test accuracy=0.513382
epoch=53600,train accuracy=0.8507223,test accuracy=0.5109489
epoch=53800,train accuracy=0.8507223,test accuracy=0.5109489
epoch=54000,train accuracy=0.8507223,test accuracy=0.5109489
epoch=54200,train accuracy=0.8507223,test accuracy=0.5182482
epoch=54400,train accuracy=0.8507223,test accuracy=0.5109489
epoch=54600,train accuracy=0.8507223,test accuracy=0.5109489
epoch=54800,train accuracy=0.8507223,test accuracy=0.5158151
epoch=55000,train accuracy=0.8507223,test accuracy=0.50851583
epoch=55200,train accuracy=0.8507223,test accuracy=0.50851583
epoch=55400,train accuracy=0.8507223,test accuracy=0.5060827
epoch=55600,train accuracy=0.8507223,test accuracy=0.513382
epoch=55800,train accuracy=0.8507223,test accuracy=0.50851583
epoch=56000,train accuracy=0.8507223,test accuracy=0.50364965
epoch=56200,train accuracy=0.8507223,test accuracy=0.49878347
epoch=56400,train accuracy=0.8507223,test accuracy=0.50851583
epoch=56600,train ac

epoch=80000,train accuracy=0.8555377,test accuracy=0.5231144
epoch=80200,train accuracy=0.8555377,test accuracy=0.52068126
epoch=80400,train accuracy=0.8555377,test accuracy=0.5182482
epoch=80600,train accuracy=0.8555377,test accuracy=0.5231144
epoch=80800,train accuracy=0.8555377,test accuracy=0.5182482
epoch=81000,train accuracy=0.8555377,test accuracy=0.5231144
epoch=81200,train accuracy=0.8555377,test accuracy=0.52068126
epoch=81400,train accuracy=0.8555377,test accuracy=0.5231144
epoch=81600,train accuracy=0.8555377,test accuracy=0.52068126
epoch=81800,train accuracy=0.8555377,test accuracy=0.52068126
epoch=82000,train accuracy=0.8555377,test accuracy=0.5231144
epoch=82200,train accuracy=0.8555377,test accuracy=0.5158151
epoch=82400,train accuracy=0.8555377,test accuracy=0.513382
epoch=82600,train accuracy=0.8555377,test accuracy=0.52068126
epoch=82800,train accuracy=0.8555377,test accuracy=0.5231144
epoch=83000,train accuracy=0.8555377,test accuracy=0.52068126
epoch=83200,train a

epoch=106800,train accuracy=0.85955054,test accuracy=0.5158151
epoch=107000,train accuracy=0.85955054,test accuracy=0.5158151
epoch=107200,train accuracy=0.85955054,test accuracy=0.5182482
epoch=107400,train accuracy=0.85955054,test accuracy=0.5182482
epoch=107600,train accuracy=0.85955054,test accuracy=0.5182482
epoch=107800,train accuracy=0.85955054,test accuracy=0.5182482
epoch=108000,train accuracy=0.85955054,test accuracy=0.5182482
epoch=108200,train accuracy=0.85955054,test accuracy=0.5158151
epoch=108400,train accuracy=0.85955054,test accuracy=0.5158151
epoch=108600,train accuracy=0.85955054,test accuracy=0.5158151
epoch=108800,train accuracy=0.85955054,test accuracy=0.5158151
epoch=109000,train accuracy=0.85955054,test accuracy=0.5158151
epoch=109200,train accuracy=0.85955054,test accuracy=0.5158151
epoch=109400,train accuracy=0.85955054,test accuracy=0.5158151
epoch=109600,train accuracy=0.85955054,test accuracy=0.5158151
epoch=109800,train accuracy=0.85955054,test accuracy=0.

epoch=133000,train accuracy=0.864366,test accuracy=0.52068126
epoch=133200,train accuracy=0.864366,test accuracy=0.5182482
epoch=133400,train accuracy=0.864366,test accuracy=0.5231144
epoch=133600,train accuracy=0.864366,test accuracy=0.5182482
epoch=133800,train accuracy=0.864366,test accuracy=0.5182482
epoch=134000,train accuracy=0.864366,test accuracy=0.52068126
epoch=134200,train accuracy=0.864366,test accuracy=0.5231144
epoch=134400,train accuracy=0.864366,test accuracy=0.5231144
epoch=134600,train accuracy=0.864366,test accuracy=0.52068126
epoch=134800,train accuracy=0.864366,test accuracy=0.5231144
epoch=135000,train accuracy=0.864366,test accuracy=0.52068126
epoch=135200,train accuracy=0.864366,test accuracy=0.5182482
epoch=135400,train accuracy=0.864366,test accuracy=0.52554744
epoch=135600,train accuracy=0.864366,test accuracy=0.5231144
epoch=135800,train accuracy=0.864366,test accuracy=0.5231144
epoch=136000,train accuracy=0.864366,test accuracy=0.5182482
epoch=136200,train 

epoch=159800,train accuracy=0.864366,test accuracy=0.52068126
epoch=160000,train accuracy=0.864366,test accuracy=0.5182482
epoch=160200,train accuracy=0.864366,test accuracy=0.5231144
epoch=160400,train accuracy=0.864366,test accuracy=0.5231144
epoch=160600,train accuracy=0.864366,test accuracy=0.52554744
epoch=160800,train accuracy=0.864366,test accuracy=0.5231144
epoch=161000,train accuracy=0.864366,test accuracy=0.5231144
epoch=161200,train accuracy=0.864366,test accuracy=0.5231144
epoch=161400,train accuracy=0.864366,test accuracy=0.5231144
epoch=161600,train accuracy=0.864366,test accuracy=0.52554744
epoch=161800,train accuracy=0.864366,test accuracy=0.5231144
epoch=162000,train accuracy=0.864366,test accuracy=0.5231144
epoch=162200,train accuracy=0.864366,test accuracy=0.5304136
epoch=162400,train accuracy=0.864366,test accuracy=0.52554744
epoch=162600,train accuracy=0.864366,test accuracy=0.52554744
epoch=162800,train accuracy=0.864366,test accuracy=0.5231144
epoch=163000,train 

epoch=186400,train accuracy=0.8691814,test accuracy=0.5158151
epoch=186600,train accuracy=0.8691814,test accuracy=0.5158151
epoch=186800,train accuracy=0.8691814,test accuracy=0.513382
epoch=187000,train accuracy=0.8691814,test accuracy=0.5109489
epoch=187200,train accuracy=0.869984,test accuracy=0.5158151
epoch=187400,train accuracy=0.8691814,test accuracy=0.513382
epoch=187600,train accuracy=0.869984,test accuracy=0.5158151
epoch=187800,train accuracy=0.869984,test accuracy=0.52068126
epoch=188000,train accuracy=0.869984,test accuracy=0.5158151
epoch=188200,train accuracy=0.869984,test accuracy=0.52068126
epoch=188400,train accuracy=0.869984,test accuracy=0.5182482
epoch=188600,train accuracy=0.869984,test accuracy=0.5158151
epoch=188800,train accuracy=0.869984,test accuracy=0.5158151
epoch=189000,train accuracy=0.869984,test accuracy=0.52068126
epoch=189200,train accuracy=0.869984,test accuracy=0.5182482
epoch=189400,train accuracy=0.869984,test accuracy=0.5182482
epoch=189600,train

epoch=213200,train accuracy=0.8707865,test accuracy=0.5158151
epoch=213400,train accuracy=0.8707865,test accuracy=0.5158151
epoch=213600,train accuracy=0.8707865,test accuracy=0.5158151
epoch=213800,train accuracy=0.8707865,test accuracy=0.5182482
epoch=214000,train accuracy=0.8707865,test accuracy=0.513382
epoch=214200,train accuracy=0.8707865,test accuracy=0.5158151
epoch=214400,train accuracy=0.87158906,test accuracy=0.513382
epoch=214600,train accuracy=0.87158906,test accuracy=0.5182482
epoch=214800,train accuracy=0.87158906,test accuracy=0.5182482
epoch=215000,train accuracy=0.87158906,test accuracy=0.5182482
epoch=215200,train accuracy=0.87158906,test accuracy=0.5231144
epoch=215400,train accuracy=0.87158906,test accuracy=0.5158151
epoch=215600,train accuracy=0.87158906,test accuracy=0.5182482
epoch=215800,train accuracy=0.87158906,test accuracy=0.5158151
epoch=216000,train accuracy=0.87158906,test accuracy=0.513382
epoch=216200,train accuracy=0.87158906,test accuracy=0.5158151
e

KeyboardInterrupt: 